### About the notebook

In this is notebook to test out ONNX based inference. Load saved model and predict succesfully. Do not worry about slow prediction. It seems first prediction is always slow, while all of the following ones are fast. Perhaps first one includes setup of objects or similar slow operations. To solve this we can just run a single empty prediction in the server right on server start.
- ```size``` - set this to your last export size matching the to_onnx export in the GoBig notebook. This will have to match with ```MODEL_SIZE``` constant in the model_server.py too. We always use square sizes, because it's easier for NNs to learn these.

In [1]:
size = 256

# ONNX inference

In [2]:
#even if we don't use the function anywhere, we need to define it, otherwise it will error out when unpickling.
def label_func(fn): return os.path.join(path,"labels",f"{fn.stem}{fn.suffix}")

In [3]:
from fastai.vision.all import PILImage, get_image_files
from fastinference.onnx import *
import torchvision.transforms as transforms
import os
import pathlib
from notebook_libs.helper_funcs import onnx_model_names

In [4]:
path = pathlib.Path().absolute()
path = os.path.join(path, "Assets", "Data~")
save_name = "unity_resnet34"+"_"+str(size)+"x"
fnames = get_image_files(os.path.join(path,"screenshots"))

Next we load the model. We can change the appended name "64x" to something else, if we saved the model via to_onnx in the GoBig notebook.

In [ ]:
#this should show pairs of .pkl and .onnx for each properly saved model via to_onnx
onnx_model_names(os.path.join(path, "models/"))

In [6]:

onnx_path = os.path.join(path, "models/", save_name+"_onnx")
onnx_learn = fastONNX(onnx_path)

using cpu: False


In [7]:
pil_img = PILImage.create(fnames[0])
resize = transforms.Resize([size, size])
pil_img = resize(pil_img)
print(pil_img.shape)
to_tensor = transforms.ToTensor()
ten_image = to_tensor(pil_img)
print(ten_image.shape)
ten_image = ten_image.unsqueeze_(0)
print(ten_image.shape)

(256, 256)
torch.Size([3, 256, 256])
torch.Size([1, 3, 256, 256])


Then we run .predict and time it. Important! This is not same predict as imported form fastinference! That one did not work for our purposes in image segmentation, so I modified it a bit to fit. You can check source with "??onnx_learn.predict" cell.

In [8]:
%%time
model_output = onnx_learn.predict(ten_image)

<class 'torch.Tensor'>  predict shape:  torch.Size([1, 3, 256, 256])
CPU times: user 331 ms, sys: 125 ms, total: 456 ms
Wall time: 955 ms


In [9]:
model_output[0].shape

(24, 256, 256)